In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['test.csv', 'train.csv', 'sample_submission.csv', 'data_description.txt']


In [2]:
data1 = pd.read_csv('../input/train.csv')
data2 = pd.read_csv('../input/test.csv')
data = pd.concat([data1, data2])

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


In [3]:
#missing values treatment
data.isnull().sum()

#make a dummy variable for NA in : alley, FireplaceQu GarageType GarageYrBlt GarageFinish GarageCars GarageArea GarageQual GarageCond 
# BsmtQual BsmtCond BsmtExposure BsmtFinType1 BsmtFinSF1 BsmtFinType2 
#since the documentation tells us that NA is supposed to happen and has an actual meaning

#lotfrontage : replace by average
data.LotFrontage.fillna(data.LotFrontage.mean(), inplace=True)

#Electrical : only one case, just take standard type for this one
data.Electrical.fillna('SBrkr', inplace = True)

#consider no garage equal to very old one
data.GarageYrBlt.fillna(data.GarageYrBlt.min(), inplace = True)

#masvnrarea has many 0 values, lets put NA to 0
data.MasVnrArea.fillna(0, inplace = True)

In [4]:
#convert the data to dummies (avoid categorical data)
data_dummy = pd.get_dummies(data, dummy_na=True)

In [5]:
#drop id column,  dont want it in the modeling data
data_dummy.drop('Id', axis = 1, inplace = True)

In [6]:
x_length = data1.shape[0]
training = data_dummy.iloc[0:x_length, :]
preding = data_dummy.iloc[x_length:, :]

In [7]:
X = training.drop('SalePrice', axis = 1).values
y = training.SalePrice.values

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [10]:
parameters = {'n_estimators': [10, 50, 100], 'max_features': [0.8, 0.9, 0.95]}
RF = GridSearchCV(RandomForestRegressor(random_state=0), parameters, cv=5)
RF.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=0, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_estimators': [10, 50, 100], 'max_features': [0.8, 0.9, 0.95]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [11]:
#mean average percent error is 0.9 -> quite good result
1 - (abs(y_test - RF.predict(X_test)) / y_test).mean()

0.9038713422485184

In [12]:
predos = RF.predict(preding.drop('SalePrice', axis = 1).fillna(0).values)
result = pd.concat([data2.Id, pd.Series(predos, name = 'SalePrice')], axis = 1)
result.to_csv('house_predos_joos.csv', index = False)